In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
import warnings
warnings.filterwarnings('ignore')


In [14]:
# Загрузим обработанный датасет, полученный после первой лабораторной (обычно .pkl формат).
df = pd.read_pickle("../data/clean_heart_disease.pkl")
print(f"Dataset shape: {df.shape}")
display(df.head())


Dataset shape: (302, 17)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,age_group,bp_category,chol_age_ratio
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1,60+,high,3.698413
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1,<40,elevated,6.756757
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1,40-50,elevated,4.975610
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1,50-60,normal,4.214286
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1,50-60,normal,6.210526


In [16]:
X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

numeric_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include='object').columns.tolist()
print(f"Numeric features: {numeric_features}")
print(f"Categorical features: {categorical_features}")


Numeric features: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'chol_age_ratio']
Categorical features: []


In [17]:
numeric_transformer = StandardScaler()
categorical_transformer = OrdinalEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features if categorical_features else [])
    ]
)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1))
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]

print("Baseline model metrics:")
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_pred_proba))


Baseline model metrics:
Precision: 0.7291666666666666
Recall: 0.8536585365853658
F1: 0.7865168539325843
ROC AUC: 0.881533101045296


In [18]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("HeartDiseaseClassification")

input_example = X_train.head(5)
signature = infer_signature(X_train.head(5), pipeline.predict(X_train.head(5)))

with mlflow.start_run(run_name="baseline_model") as run:
    mlflow.log_param("model_type", "RandomForest")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("preprocessor", "StandardScaler, OrdinalEncoder")
    mlflow.log_metric("precision", precision_score(y_test, y_pred))
    mlflow.log_metric("recall", recall_score(y_test, y_pred))
    mlflow.log_metric("f1", f1_score(y_test, y_pred))
    mlflow.log_metric("roc_auc", roc_auc_score(y_test, y_pred_proba))
    mlflow.sklearn.log_model(
        sk_model=pipeline,
        artifact_path="model",
        signature=signature,
        input_example=input_example
    )
    print(f"Run ID: {run.info.run_id}")


2025/11/09 20:39:19 INFO mlflow.tracking.fluent: Experiment with name 'HeartDiseaseClassification' does not exist. Creating a new experiment.


2025/11/09 20:39:29 INFO mlflow.tracking._tracking_service.client: 🏃 View run baseline_model at: http://localhost:5000/#/experiments/1/runs/e8ebdd8b313b444c8a740d67dbbf5722.
2025/11/09 20:39:29 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.


Run ID: e8ebdd8b313b444c8a740d67dbbf5722


In [19]:
from sklearn.preprocessing import PolynomialFeatures, KBinsDiscretizer

# Выберите 2-3 числовых признака для расширения
numeric_for_poly = numeric_features[:2]
poly_transformer = PolynomialFeatures(degree=2, include_bias=False)

numeric_for_bins = numeric_features[:3]
kbin_transformer = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='quantile')

extended_preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numeric_features),
    ('cat', OrdinalEncoder(), categorical_features if categorical_features else []),
    ('poly', Pipeline([
        ('poly', poly_transformer),
        ('scaler', StandardScaler())
    ]), numeric_for_poly),
    ('kbin', kbin_transformer, numeric_for_bins),
])

X_train_fe = extended_preprocessor.fit_transform(X_train, y_train)
X_test_fe = extended_preprocessor.transform(X_test)


In [20]:
extended_model = Pipeline([
    ('preprocessor', extended_preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1))
])

extended_model.fit(X_train, y_train)
y_pred_ext = extended_model.predict(X_test)
y_pred_proba_ext = extended_model.predict_proba(X_test)[:, 1]

print("Extended features model metrics:")
print("Precision:", precision_score(y_test, y_pred_ext))
print("Recall:", recall_score(y_test, y_pred_ext))
print("F1:", f1_score(y_test, y_pred_ext))
print("ROC AUC:", roc_auc_score(y_test, y_pred_proba_ext))


Extended features model metrics:
Precision: 0.8
Recall: 0.8780487804878049
F1: 0.8372093023255814
ROC AUC: 0.894773519163763


In [21]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("HeartDiseaseClassification")

input_example_ext = X_train.head(5)
signature_ext = infer_signature(X_train.head(5), extended_model.predict(X_train.head(5)))

with mlflow.start_run(run_name="sklearn_extended_features") as run:
    mlflow.log_param("model_type", "RandomForest")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("preprocessor", "StandardScaler, OrdinalEncoder, PolynomialFeatures, KBinsDiscretizer")
    mlflow.log_metric("precision", precision_score(y_test, y_pred_ext))
    mlflow.log_metric("recall", recall_score(y_test, y_pred_ext))
    mlflow.log_metric("f1", f1_score(y_test, y_pred_ext))
    mlflow.log_metric("roc_auc", roc_auc_score(y_test, y_pred_proba_ext))
    mlflow.sklearn.log_model(
        sk_model=extended_model,
        artifact_path="model",
        signature=signature_ext,
        input_example=input_example_ext
    )
    print(f"Run ID: {run.info.run_id}")


2025/11/09 20:43:10 INFO mlflow.tracking._tracking_service.client: 🏃 View run sklearn_extended_features at: http://localhost:5000/#/experiments/1/runs/06a435a0b9614b71933dc081a6ee35d7.
2025/11/09 20:43:10 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.


Run ID: 06a435a0b9614b71933dc081a6ee35d7


In [22]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.base import BaseEstimator, TransformerMixin

# Определим, сколько признаков оставить (например, 30% от общего числа)
n_features_to_select = max(1, int(X_train_fe.shape[1] * 0.3))

sfs = SFS(
    estimator=RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1),
    k_features=n_features_to_select,
    forward=True,
    floating=False,
    scoring='f1',
    cv=3,
    n_jobs=-1
)
sfs.fit(X_train_fe, y_train)
selected_idx = list(sfs.k_feature_idx_)
print("Selected feature indices:", selected_idx)


Selected feature indices: [2, 6, 10, 11, 12, 15]


In [23]:
# Вспомогательный класс для выбора столбцов по индексам
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_indices):
        self.feature_indices = feature_indices
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[:, self.feature_indices]

# Построение пайплайна
selected_model = Pipeline([
    ('preprocessor', extended_preprocessor),
    ('featureselector', FeatureSelector(selected_idx)),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1))
])
selected_model.fit(X_train, y_train)
y_pred_sel = selected_model.predict(X_test)
y_pred_proba_sel = selected_model.predict_proba(X_test)[:, 1]

print("Feature selection model metrics:")
print("Precision:", precision_score(y_test, y_pred_sel))
print("Recall:", recall_score(y_test, y_pred_sel))
print("F1:", f1_score(y_test, y_pred_sel))
print("ROC AUC:", roc_auc_score(y_test, y_pred_proba_sel))

# Логирование в MLflow
input_example_sel = X_train.head(5)
signature_sel = infer_signature(X_train.head(5), selected_model.predict(X_train.head(5)))

with mlflow.start_run(run_name="sfs_selected_features") as run:
    mlflow.log_param("model_type", "RandomForest")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("preprocessor", "StandardScaler, OrdinalEncoder, PolynomialFeatures, KBinsDiscretizer, SequentialFeatureSelector")
    mlflow.log_param("selected_features_count", len(selected_idx))
    mlflow.log_metric("precision", precision_score(y_test, y_pred_sel))
    mlflow.log_metric("recall", recall_score(y_test, y_pred_sel))
    mlflow.log_metric("f1", f1_score(y_test, y_pred_sel))
    mlflow.log_metric("roc_auc", roc_auc_score(y_test, y_pred_proba_sel))
    mlflow.sklearn.log_model(
        sk_model=selected_model,
        artifact_path="model",
        signature=signature_sel,
        input_example=input_example_sel
    )
    print(f"Run ID: {run.info.run_id}")


Feature selection model metrics:
Precision: 0.7659574468085106
Recall: 0.8780487804878049
F1: 0.8181818181818182
ROC AUC: 0.8825783972125436


2025/11/09 20:46:56 INFO mlflow.tracking._tracking_service.client: 🏃 View run sfs_selected_features at: http://localhost:5000/#/experiments/1/runs/62e61eccdb104bc086fdad309f066d2d.
2025/11/09 20:46:56 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.


Run ID: 62e61eccdb104bc086fdad309f066d2d


In [24]:
import optuna
from sklearn.model_selection import cross_val_score

# Используем расширенный пайплайн с отбором признаков

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'max_depth': trial.suggest_int('max_depth', 5, 15),
        'max_features': trial.suggest_float('max_features', 0.1, 1.0),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 8),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 4)
    }
    model = Pipeline([
        ('preprocessor', extended_preprocessor),
        ('featureselector', FeatureSelector(selected_idx)),
        ('classifier', RandomForestClassifier(**params, random_state=42, n_jobs=-1))
    ])
    # Кросс-валидация по f1 для классификации
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='f1', n_jobs=-1)
    return score.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10) # Минимум 10 прогонов

print('Best trial:')
print(study.best_trial)
print('Best parameters:', study.best_params)


[I 2025-11-09 20:48:01,244] A new study created in memory with name: no-name-d796c9ef-020f-4d99-b046-8a315b7a5be2
[I 2025-11-09 20:48:03,540] Trial 0 finished with value: 0.8484320557491288 and parameters: {'n_estimators': 111, 'max_depth': 14, 'max_features': 0.24114919722089756, 'min_samples_split': 6, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.8484320557491288.
[I 2025-11-09 20:48:05,636] Trial 1 finished with value: 0.8567257945306727 and parameters: {'n_estimators': 128, 'max_depth': 11, 'max_features': 0.6397791857030557, 'min_samples_split': 6, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.8567257945306727.
[I 2025-11-09 20:48:06,362] Trial 2 finished with value: 0.842885872297637 and parameters: {'n_estimators': 66, 'max_depth': 8, 'max_features': 0.8981071585366388, 'min_samples_split': 8, 'min_samples_leaf': 2}. Best is trial 1 with value: 0.8567257945306727.
[I 2025-11-09 20:48:07,290] Trial 3 finished with value: 0.8569844789356984 and parameters: {'n_est

Best trial:
FrozenTrial(number=4, state=TrialState.COMPLETE, values=[0.8614275282949982], datetime_start=datetime.datetime(2025, 11, 9, 20, 48, 7, 291503), datetime_complete=datetime.datetime(2025, 11, 9, 20, 48, 8, 784259), params={'n_estimators': 178, 'max_depth': 14, 'max_features': 0.6286174609667049, 'min_samples_split': 2, 'min_samples_leaf': 4}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=200, log=False, low=50, step=1), 'max_depth': IntDistribution(high=15, log=False, low=5, step=1), 'max_features': FloatDistribution(high=1.0, log=False, low=0.1, step=None), 'min_samples_split': IntDistribution(high=8, log=False, low=2, step=1), 'min_samples_leaf': IntDistribution(high=4, log=False, low=1, step=1)}, trial_id=4, value=None)
Best parameters: {'n_estimators': 178, 'max_depth': 14, 'max_features': 0.6286174609667049, 'min_samples_split': 2, 'min_samples_leaf': 4}


In [27]:
# Обучаем лучшую модель на полном датасете
best_params = study.best_params

production_model = Pipeline([
    ('preprocessor', extended_preprocessor),
    ('featureselector', FeatureSelector(selected_idx)),
    ('classifier', RandomForestClassifier(**best_params, random_state=42, n_jobs=-1))
])
production_model.fit(X, y)

# Получаем сигнатуру и пример входных данных
input_example_full = X.head(5)
signature_full = infer_signature(X.head(5), production_model.predict(X.head(5)))

# Логирование Production модели
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("HeartDiseaseClassification")

with mlflow.start_run(run_name="production_model") as run:
    mlflow.log_param("model_type", "RandomForest")
    for key, value in best_params.items():
        mlflow.log_param(key, value)
    mlflow.log_param("training_set", "full")
    mlflow.log_param("selected_features_count", len(selected_idx))

    mlflow.sklearn.log_model(
        sk_model=production_model,
        artifact_path="model",
        signature=signature_full,
        input_example=input_example_full
    )

    print(f"Production Run ID: {run.info.run_id}")


2025/11/09 21:26:57 INFO mlflow.tracking._tracking_service.client: 🏃 View run production_model at: http://localhost:5000/#/experiments/1/runs/53cbd22b6ce74c6cb75023c10374863c.
2025/11/09 21:26:57 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.


Production Run ID: 53cbd22b6ce74c6cb75023c10374863c
